In [17]:
import pandas as pd
import numpy as np
from datetime import date
import regex as re
from datetime import datetime, timedelta

In [23]:
# import data
indeed_df = pd.read_csv(r"C:\Users\bosst\Data Science Job Market Research\data\indeed_jobs.csv")
glassdoor_df = pd.read_csv(r"C:\Users\\bosst\Data Science Job Market Research\data\glassdoor_jobs.csv")

In [24]:
def clean_date_indeed(date_str):
    date_str = date_str.split(' ')
    if (date_str[0] == "Posted\nJust") or (date_str[-1] == "Posted\nToday"):
        return date.today()
    if (date_str[0] == "Posted\nPosted"):
        post_day = timedelta(days=(int(date_str[1].replace('+',''))), hours=0)
        return date.today() - post_day
    # if posting are not active, return the date minus 30 day,
    if date_str[0] == "Employer\nActive":
        post_day = timedelta(days=(int(date_str[1].replace('+','')) + 30), hours=0)
        return date.today() - post_day
indeed_df['date'] = indeed_df['date'].apply(clean_date_indeed)

In [33]:
def get_pay_indeed(pay):
    if pay is np.nan: return None
        # remove word 'an','a','From' string
    pay = pay.replace("From ","").replace(" an","").replace(" a","")
    # remove character ',' and '$' from string
    pay = pay.replace("$" , "").replace(",","")
    # replace '-' with space\n",
    pay = pay.replace("–" , " ")
    convert_dic = {'year' : 1,'month' : 12,'hour' : 2087}
    pay = pay.split(' ')
    # if only a value of pay given
    if len(pay) == 2:
        pay_value = float(pay[0])
        convert_rate = convert_dic[pay[-1]]
        return str(pay_value * convert_rate)
    # if a range of pay given
    elif len(pay) == 3:
        pay_value_start = float(pay[0])
        pay_value_end = float(pay[1])
        convert_rate = convert_dic[pay[-1]]
        return str(pay_value_start * convert_rate) + ' - ' +  str(pay_value_end * convert_rate)
indeed_df['pay'] = indeed_df['pay'].apply(get_pay_indeed)

In [36]:
location = indeed_df['company_location'][10]
def clean_location_indeed(location):
    street_address = None
    city = None
    province = None
    remote = False
    hybrid = False
    # replace character "•" with ", " and split by ", " n",
    location = location.replace("•",", ").split(", ")
    # check if the job is fully remote
    if (location[-1] == "Remote") or (location[-1] == "Hybrid remote"):
        remote = True
    if location[-1] == "Hybrid remote":
        hybrid = True
    length_location = len(location)
    # extract province and city\n",
    list_province = ["AB", "BC", "MB", "NB", "NL", "NS", "NT", "NU", "ON", "PE", "QC", "SK", "YT"]
    for text in list(reversed(location)):
        if (province is not None) and (city is not None) and (street_address is None):
            street_address = text
        if (province is not None) and (city is None):
            city = text
        if text in list_province:
            province = text
    return street_address,city,province,remote,hybrid
indeed_df['company_location'] = indeed_df['company_location'].apply(clean_location_indeed)

In [40]:
# create new columns in indeef_df to store location: street_address,city,province,remote,hybrid
indeed_df['address'] = indeed_df['company_location'].apply(lambda x: x[0])
indeed_df['city'] = indeed_df['company_location'].apply(lambda x: x[1])
indeed_df['province'] = indeed_df['company_location'].apply(lambda x: x[2])
indeed_df['remote'] = indeed_df['company_location'].apply(lambda x: x[3])
indeed_df['hybrid'] = indeed_df['company_location'].apply(lambda x: x[4])

In [41]:
# clean Indead job type
# line break is replaced with comma
indeed_df['job_type'] = indeed_df['job_type'].str.replace("\n",",")

In [43]:
# clean date
def clean_date_glassdoor(post_age):
    if post_age[-1] == 'h':
        return date.today()
   # remove '+' and space from date
    post_age = post_age.replace("d","").replace("+","")
    return date.today() - timedelta(days = int(post_age))
glassdoor_df['posting_age'] = glassdoor_df['posting_age'].apply(clean_date_glassdoor)
# change name columns "posting_age" to "date"
glassdoor_df = glassdoor_df.rename(columns = {"posting_age":"date"})

In [46]:
# clean pay
def clean_pay_glassdoor(pay):
    pay = pay.replace("$","").replace("/","").replace(" (est.)","").replace(",","")
    rate_label = pay[-2:]
    pay = float(pay[:-2])
    convert_dic = {'yr' : 1,'mo' : 12,'hr' : 2087}
    return pay * convert_dic[rate_label]
glassdoor_df['pay'] = glassdoor_df['pay'].apply(clean_pay_glassdoor)

In [47]:
glassdoor_df

,date,job_title,company_name,location,pay,job_description
0,2023-11-11,Développeur Python (Machine Learning/NLP/LLM) ...,SIA Innovations Inc.,Montreal,83378.00,Seulement les candidats qui vivent à Montréal ...
1,2023-11-25,Junior Data Scientist,Theory and Practice,Vancouver,111030.00,Description\nThe Junior Data Scientist will jo...
2,2023-12-11,"Director of Algorithms, Modelling & Statistica...",Vancouver Highlanders Rugby,Vancouver,75000.00,VANCOUVER HIGHLANDERSRUGBY IS LOOKING FOR THEI...
3,2023-11-20,Data Science Intern - Montreal,"Analysis Group, Inc.",Montreal,89644.00,Overview:\nAnalysis Group is one of the larges...
4,2023-11-11,Data Science Intern- Summer 2024 (Canada),Altair Engineering,Toronto,117983.00,Transforming the Future with the Convergence o...
...,...,...,...,...,...,...
505,2023-12-04,DECISION SUPPORT SPECIALIST,St. Mary’s General Hospital,Kitchener,105205.67,Position Description: Temporary Full Time Hybr...
506,2023-11-11,Statistical Science Associate Director/Senior ...,AstraZeneca,Mississauga,105205.67,"At AstraZeneca, we pride ourselves on crafting..."
507,2023-11-11,AI and Machine Learning Principal,Apply Digital,Toronto,122034.00,Who we are: We’re a global digital transformat...
508,2023-11-11,Senior Machine Learning Engineer (Remote),OpenTable,Vancouver,114896.00,"With millions of diners, tens of thousands of ..."


In [52]:
# clean job location
def clean_location_glassdoor(location):
    remote = False
    city = location
    province = None
    if location == "Remote" or location == "Canada":
        remote = True
        city = None
    provinces = ['Alberta', 'British Columbia', 'Manitoba', 'New Brunswick', 'Newfoundland And Labrador', 'Nova Scotia', 'Ontario', 'Prince Edward Island', 'Quebec', 'Saskatchewan']
    if location in provinces:
        city = None
        province = location
    return city, province, remote
glassdoor_df['location'] = glassdoor_df['location'].apply(clean_location_glassdoor)


In [53]:
glassdoor_df

,date,job_title,company_name,location,pay,job_description
0,2023-11-11,Développeur Python (Machine Learning/NLP/LLM) ...,SIA Innovations Inc.,"(Montreal, None, False)",83378.00,Seulement les candidats qui vivent à Montréal ...
1,2023-11-25,Junior Data Scientist,Theory and Practice,"(Vancouver, None, False)",111030.00,Description\nThe Junior Data Scientist will jo...
2,2023-12-11,"Director of Algorithms, Modelling & Statistica...",Vancouver Highlanders Rugby,"(Vancouver, None, False)",75000.00,VANCOUVER HIGHLANDERSRUGBY IS LOOKING FOR THEI...
3,2023-11-20,Data Science Intern - Montreal,"Analysis Group, Inc.","(Montreal, None, False)",89644.00,Overview:\nAnalysis Group is one of the larges...
4,2023-11-11,Data Science Intern- Summer 2024 (Canada),Altair Engineering,"(Toronto, None, False)",117983.00,Transforming the Future with the Convergence o...
...,...,...,...,...,...,...
505,2023-12-04,DECISION SUPPORT SPECIALIST,St. Mary’s General Hospital,"(Kitchener, None, False)",105205.67,Position Description: Temporary Full Time Hybr...
506,2023-11-11,Statistical Science Associate Director/Senior ...,AstraZeneca,"(Mississauga, None, False)",105205.67,"At AstraZeneca, we pride ourselves on crafting..."
507,2023-11-11,AI and Machine Learning Principal,Apply Digital,"(Toronto, None, False)",122034.00,Who we are: We’re a global digital transformat...
508,2023-11-11,Senior Machine Learning Engineer (Remote),OpenTable,"(Vancouver, None, False)",114896.00,"With millions of diners, tens of thousands of ..."


In [54]:
glassdoor_df['city'] = glassdoor_df['location'].apply(lambda x: x[0])
glassdoor_df['remote'] = glassdoor_df['location'].apply(lambda x: x[2])
glassdoor_df['province'] = glassdoor_df['location'].apply(lambda x: x[1])

In [56]:
# join data from 2 job boards, some unnecessary columns are not selected
indeed_df = indeed_df[['job_title','date','company_name','address','city','province','remote','hybrid','pay','job_description','benefits','job_type']]
glassdoor_df = glassdoor_df[['job_title','date','company_name','city','remote','pay','job_description']]
job_postings_df = pd.concat([indeed_df,glassdoor_df], axis = 0)

In [58]:
# fill all None cell with np.nan
job_postings_df = job_postings_df.fillna(value = np.nan)
# drop duplicate row with same company, job title and date post
job_postings_df = job_postings_df.drop_duplicates(subset=['job_title','company_name','date'], keep='first').reset_index().drop(['index'], axis = 1)
# drop row with empty job description
job_postings_df = job_postings_df[~job_postings_df['job_description'].isna()]

In [61]:
job_postings_df

,job_title,date,company_name,address,city,province,remote,hybrid,pay,job_description,benefits,job_type
0,Machine Learning Engineer,2023-12-11,Blurb.fm,NaN,NaN,NaN,True,False,66613.42 - 144330.8,ML ENGINEER\nAre you a machine learning expert...,NaN,Fixed term contract
1,"Ingénieur de données et visualisation, Science...",2023-12-11,Pratt & Whitney,NaN,Longueuil,QC,False,False,66613.42 - 144330.8,Date Posted:\n2023-11-14\nCountry:\nCanada\nLo...,"['Company pension', 'Flexible schedule']",Full-time
2,"Data Analyst, Merchandise Planning",2023-12-11,SSENSE,NaN,Montréal,QC,False,False,66613.42 - 144330.8,"Company Description\n\nSSENSE, pronounced [es-...","['Company pension', 'Flexible schedule']",Fixed term contract
3,Data Science / Machine learning specialist - F...,2023-12-11,CIUSSS de l'Ouest-de-l’Île-de-Montréal,NaN,Verdun,QC,False,False,66613.42 - 144330.8,Reason for Post :\nReplacement more than de 6 ...,"['Company pension', 'Flexible schedule']",Full-time
4,Spécialiste en science des données / apprentis...,2023-12-11,CIUSSS de l'Ouest-de-l’Île-de-Montréal,NaN,Verdun,QC,False,False,66613.42 - 144330.8,Raison d'être de l'affichage :\nRemplacement d...,"['Company pension', 'Flexible schedule']","Temporary,Full-time"
...,...,...,...,...,...,...,...,...,...,...,...,...
631,Sr. Computer Vision / ML Engineer,2023-11-11,Potential Motors,NaN,Fredericton,NaN,False,NaN,130000.0,You’re passionate about a sustainable future a...,NaN,NaN
632,"Machine Learning Engineer MLOps/MLaaS, ICC AI ...",2023-11-11,Amazon Development Centre Canada ULC,NaN,Vancouver,NaN,False,NaN,114800.0,2+ years of non-internship professional softwa...,NaN,NaN
633,AI and Machine Learning Principal,2023-11-11,Apply Digital,NaN,Toronto,NaN,False,NaN,122034.0,Who we are: We’re a global digital transformat...,NaN,NaN
634,Senior Machine Learning Engineer (Remote),2023-11-11,OpenTable,NaN,Vancouver,NaN,False,NaN,114896.0,"With millions of diners, tens of thousands of ...",NaN,NaN


In [63]:
city_dic = {
    "Airdrie": "AB",
    "Calgary": "AB",
    "Camrose": "AB",
    "Cold Lake": "AB",
    "Edmonton": "AB",  # AB Provincial capital
    "Fort Saskatchewan": "AB",
    "Grande Prairie": "AB",
    "Leduc": "AB",
    "Lethbridge": "AB",
    "Lloydminster": "AB",
    "Medicine Hat": "AB",
    "Red Deer": "AB",
    "Spruce Grove": "AB",
    "St. Albert": "AB",
    "Westaskiwin": "AB",
    "Abbotsford": "BC",
    "Armstrong": "BC",
    "Burnaby": "BC",
    "Castlegar": "BC",
    "Chilliwack": "BC",
    "Colwood": "BC",
    "Coquitlam": "BC",
    "Courtenay": "BC",
    "Cranbrook": "BC",
    "Dawson Creek": "BC",
    "Duncan": "BC",
    "Enderby": "BC",
    "Fernie": "BC",
    "Fort St. John": "BC",
    "Grand Forks": "BC",
    "Greenwood": "BC",
    "Kamloops": "BC",
    "Kimberley": "BC",
    "Kelowna": "BC",
    "Langley": "BC",
    "Merritt": "BC",
    "Nanaimo": "BC",
    "Nelson": "BC",
    "New Westminster": "BC",
    "North Vancouver": "BC",
    "Parksville": "BC",
    "Penticton": "BC",
    "Port Alberni": "BC",
    "Port Coquitlam": "BC",
    "Port Moody": "BC",
    "Prince George": "BC",
    "Prince Rupert": "BC",
    "Quesnel": "BC",
    "Revelstoke": "BC",
    "Richmond": "BC",
    "Rossland": "BC",
    "Salmon Arm": "BC",
    "Surrey": "BC",
    "Terrace": "BC",
    "Trail": "BC",
    "Vancouver": "BC",
    "Vernon": "BC",
    "Victoria": "BC",  # Provincial capital
    "White Rock": "BC",
    "Williams Lake": "BC",
    "Brandon": "MB",
    "Dauphin": "MB",
    "Flin Flon": "MB",  # Partly in Saskatchewan
    "Portage la Prairie": "MB",
    "Selkirk": "MB",
    "Steinbach": "MB",
    "Thompson": "MB",
    "Winkler": "MB",
    "Winnipeg": "MB",  # Provincial capital
    "Bathurst": "NB",
    "Campbellton": "NB",
    "Dieppe": "NB",
    "Edmundston": "NB",
    "Fredericton": "NB",  # Provincial capital
    "Miramichi": "NB",
    "Moncton": "NB",
    "Saint John": "NB",
    "Newfoundland and Labrador": "NL",
    "Corner Brook": "NL",
    "Mount Pearl": "NL",
    "St. John's": "NL",  # Provincial capital
    "Halifax": "NS",  # Provincial capital, now part of the Regional Municipality of Halifax
    "Sydney": "NS",
    "Barrie": "ON",
    "Belleville": "ON",
    "Brampton": "ON",
    "Brant": "ON",
    "Brantford": "ON",
    "Brockville": "ON",
    "Burlington": "ON",
    "Cambridge": "ON",
    "Chatham-Kent": "ON",
    "Clarence-Rockland": "ON",
    "Cornwall": "ON",
    "Dryden": "ON",
    "Elliot Lake": "ON",
    "Greater Sudbury": "ON",
    "Guelph": "ON",
    "Haldimand County": "ON",
    "Hamilton": "ON",
    "Kawartha Lakes": "ON",
    "Kenora": "ON",
    "Kingston": "ON",
    "Kitchener": "ON",
    "Lambton Shores": "ON",
    "London": "ON",
    "Mississauga": "ON",
    "Niagara Falls": "ON",
    "Norfolk County": "ON",
    "North Bay": "ON",
    "Orillia": "ON",
    "Oshawa": "ON",
    "Ottawa": "ON",  # National capital
    "Owen Sound": "ON",
    "Pembroke": "ON",
    "Peterborough": "ON",
    "Pickering": "ON",
    "Prince Edward County": "ON",
    "Port Colborne": "ON",
    "Quinte West": "ON",
    "Sarnia": "ON",
    "Sault Ste. Marie": "ON",
    "St. Catharines": "ON",
    "St. Thomas": "ON",
    "Stratford": "ON",
    "Temiskaming Shores": "ON",
    "Thorold": "ON",
    "Thunder Bay": "ON",
    "Timmins": "ON",
    "Toronto": "ON",  # Provincial capital
    "Vaughan": "ON",
    "Waterloo": "ON",
    "Welland": "ON",
    "Windsor": "ON",
    "Woodstock": "ON",
    "Montreal": "QC",
    "Quebec": "QC",  # Provincial capital
    "Charlottetown": "PE",  # Provincial capital
    "Summerside": "PE",
    "Gatineau": "QC",
    "Sherbrooke": "QC",
    "Estevan": "SK",
    "Flin Flon": "SK",  # Mostly in Manitoba
    "Humboldt": "SK",
    "Lloydminster": "SK",  # Partly in Alberta
    "Melfort": "SK",
    "Melville": "SK",
    "Moose Jaw": "SK",
    "North Battleford": "SK",
    "Prince Albert": "SK",
    "Regina": "SK",  # Provincial capital
    "Saskatoon": "SK",
    "Swift Current": "SK",
    "Weyburn": "SK",
    "Yorkton": "SK",
    "Yellowknife": "NT",
    "Iqaluit": "NU",  # Territorial capital
    "Whitehorse": "YT",  # Territorial capital
    "St. Jacobs": "ON",
    "Victoria West": "BC",
    "Etobicoke": "ON",
    "Milton": "ON",
    "Saguenay": "QC",
    "Baie-d'Urfé": "QC",
}
for index in job_postings_df['province'].index:
    if pd.isna(job_postings_df['province'][index]) or pd.isna(job_postings_df['city'][index]): continue
    city_dic[job_postings_df['city'][index]] = job_postings_df['province'][index]
for index in job_postings_df['province'].index:
    if pd.isna(job_postings_df['province'][index]):
        if (not pd.isna(job_postings_df['city'][index])) and (job_postings_df['city'][index] in city_dic):
            job_postings_df['province'][index] = city_dic[job_postings_df['city'][index]]

In [64]:
job_postings_df

,job_title,date,company_name,address,city,province,remote,hybrid,pay,job_description,benefits,job_type
0,Machine Learning Engineer,2023-12-11,Blurb.fm,NaN,NaN,NaN,True,False,66613.42 - 144330.8,ML ENGINEER\nAre you a machine learning expert...,NaN,Fixed term contract
1,"Ingénieur de données et visualisation, Science...",2023-12-11,Pratt & Whitney,NaN,Longueuil,QC,False,False,66613.42 - 144330.8,Date Posted:\n2023-11-14\nCountry:\nCanada\nLo...,"['Company pension', 'Flexible schedule']",Full-time
2,"Data Analyst, Merchandise Planning",2023-12-11,SSENSE,NaN,Montréal,QC,False,False,66613.42 - 144330.8,"Company Description\n\nSSENSE, pronounced [es-...","['Company pension', 'Flexible schedule']",Fixed term contract
3,Data Science / Machine learning specialist - F...,2023-12-11,CIUSSS de l'Ouest-de-l’Île-de-Montréal,NaN,Verdun,QC,False,False,66613.42 - 144330.8,Reason for Post :\nReplacement more than de 6 ...,"['Company pension', 'Flexible schedule']",Full-time
4,Spécialiste en science des données / apprentis...,2023-12-11,CIUSSS de l'Ouest-de-l’Île-de-Montréal,NaN,Verdun,QC,False,False,66613.42 - 144330.8,Raison d'être de l'affichage :\nRemplacement d...,"['Company pension', 'Flexible schedule']","Temporary,Full-time"
...,...,...,...,...,...,...,...,...,...,...,...,...
631,Sr. Computer Vision / ML Engineer,2023-11-11,Potential Motors,NaN,Fredericton,NB,False,NaN,130000.0,You’re passionate about a sustainable future a...,NaN,NaN
632,"Machine Learning Engineer MLOps/MLaaS, ICC AI ...",2023-11-11,Amazon Development Centre Canada ULC,NaN,Vancouver,BC,False,NaN,114800.0,2+ years of non-internship professional softwa...,NaN,NaN
633,AI and Machine Learning Principal,2023-11-11,Apply Digital,NaN,Toronto,ON,False,NaN,122034.0,Who we are: We’re a global digital transformat...,NaN,NaN
634,Senior Machine Learning Engineer (Remote),2023-11-11,OpenTable,NaN,Vancouver,BC,False,NaN,114896.0,"With millions of diners, tens of thousands of ...",NaN,NaN


In [66]:
job_postings_df.to_csv(r"C:\Users\bosst\Data Science Job Market Research\data\job_postings.csv", index = False)